In [3]:
import os
import warnings
warnings.filterwarnings("ignore")

In [5]:
from get_rainfall_data import load_chirps_data

In [6]:
import pandas as pd
import numpy as np
import xarray as xr
import rasterio as rio
import rioxarray
from shapely.geometry import shape
import seaborn as sns
import geopandas as gpd

In [7]:
from s2cloudless import S2PixelCloudDetector
from odc.algo import to_rgba
from odc.stac import stac_load, configure_rio
import dask
from deafrica_tools.plotting import rgb, display_map
from pystac_client import Client

-  Digital Earth Africa data is stored on S3 in Cape Town, Africa. 
- To load the data, we must configure rasterio with the appropriate AWS S3 endpoint.

In [5]:
os.environ["AWS_S3_ENDPOINT"] = "s3.af-south-1.amazonaws.com"
os.environ["AWS_NO_SIGN_REQUEST"] = "true"

- The configuration below must be used when loading any Digital Earth Africa data through the STAC API.

In [6]:
client = dask.distributed.Client()

configure_rio(
    cloud_defaults = True,
    aws = {"aws_unsigned": True},
    client = client
)

- Open the stac catalog

In [7]:
catalog = Client.open("https://explorer.digitalearth.africa/stac")

#### Load Rainfal Data for a farmer in Kisumu County

In [8]:
## Get fields data:
field = gpd.read_file('dr_okello_field.geojson',crs='EPSG:4326')

In [9]:
round(field.centroid.geometry.x[0],3)

34.855

In [10]:
lat = round(field.centroid.geometry.y[0],3)
lon = round(field.centroid.geometry.x[0],3)

In [11]:
lat,lon

(-0.044, 34.855)

In [12]:
start_date = '2023-01-01'
end_date = '2023-08-01'

In [13]:
rainfall_data = load_chirps_data(lat, lon, start_date,end_date)

--- Found:  212 datasets ---


In [14]:
rainfall_data

<xarray.Dataset>
Dimensions:      (time: 7, latitude: 1600, longitude: 1500)
Coordinates:
  * latitude     (latitude) float64 39.98 39.93 39.88 ... -39.88 -39.93 -39.98
  * longitude    (longitude) float64 -19.98 -19.93 -19.88 ... 54.88 54.93 54.98
    spatial_ref  int32 4326
  * time         (time) datetime64[ns] 2023-01-15 2023-02-15 ... 2023-07-15
Data variables:
    rainfall     (time, latitude, longitude) float32 dask.array<chunksize=(1, 1600, 1500), meta=np.ndarray>

In [42]:
rainfall_data = rainfall_data.compute()

In [19]:
rainfall_ds = rainfall_data.compute()

In [20]:
rainfall_ds

<xarray.Dataset>
Dimensions:      (time: 5, latitude: 0, longitude: 1)
Coordinates:
  * latitude     (latitude) float64 
  * longitude    (longitude) float64 34.88
    spatial_ref  int32 4326
  * time         (time) datetime64[ns] 2023-03-15 2023-04-15 ... 2023-07-15
Data variables:
    rainfall     (time, latitude, longitude) float32

#### Get S2 Data between January to September for field